In [1]:
import pandas as pd
import os
import numpy as np
from IPython.display import display, HTML
import utils
import matplotlib.pyplot as plt
import tensorflow as tf
import math
plt.rcParams.update({'font.size': 15})
plt.style.use('seaborn-white')

gpus = tf.config.experimental.list_physical_devices('GPU')

tf.config.experimental.set_visible_devices(gpus[1], 'GPU')
tf.compat.v1.disable_eager_execution()

In [2]:
# Elegir que experimento evaluar
NUM_CLASSES = 7 
directorio_experimento = f'./{NUM_CLASSES}_class'

os.chdir(directorio_experimento)
reporte_folder = f'reporte_{NUM_CLASSES}_class'
if not os.path.exists(reporte_folder):
    os.mkdir(reporte_folder)

In [3]:
# datos = pd.read_csv('/database/bd_aumentada/database_augment.csv')
df_rendimiento = pd.read_csv('rendimiento_experimento.csv')
# cambiar para el numero de clases
# class_list = datos[f'Class_cat_{NUM_CLASSES}'].unique().tolist()
print('El rendimiento del experimento fue: ')
display(df_rendimiento)
desc = df_rendimiento[['acc', 'loss']].describe()
desc.to_csv('describe_experimento.csv')
display(desc)
print(f'La precision total fue de: {df_rendimiento.acc.mean()} (+/- {df_rendimiento.acc.std()})')

In [4]:
# Elegimos el mejor elemento
best_index = df_rendimiento.acc.idxmax()
best_folder = f'fold_{best_index}'
print(f'El mejor rendimiento fue encontrado en el: {best_folder}')

rendimiento_fold = df_rendimiento.acc.max()
print(rendimiento_fold)
df_log = pd.read_csv(os.path.join(best_folder, 'log.csv'))
display(df_log)
acc = df_log.accuracy.values
val_accuracy = df_log.val_accuracy.values

loss = df_log.loss.values
val_loss = df_log.val_loss.values
ep = range(len(acc))
mejor = df_log.val_accuracy.idxmax()
display(df_log.iloc[mejor])

In [5]:
if mejor > rendimiento_fold:
    model_file = 'model.best.h5'
else:
    model_file = 'model.h5'
    
BATCH_SIZE = 256
WIDTH = 256 # 256
HEIGHT = 256 # 256
SEED = 111091
    
model = tf.keras.models.load_model(f'./{best_folder}/{model_file}')

# model = tf.saved_model.load(os.path.join(best_folder, 'vgg19_multiclass'))
datos_val = pd.read_csv(os.path.join(best_folder, 'datos_val.csv'))
    
test_datagen =tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.vgg19.preprocess_input,
)

test_generator = test_datagen.flow_from_dataframe(datos_val, 
                                      None, 
                                      x_col='file', 
                                      target_size=(WIDTH, HEIGHT),
                                      y_col=f'Class_cat_{NUM_CLASSES}', 
                                      batch_size=BATCH_SIZE,
                                      seed=SEED,
                                      has_ext=True,
                                      class_mode='categorical', 
                                      shuffle=False)
class_list = list(test_generator.class_indices.keys())
print('Validando')
Y_pred = model.predict(test_generator, verbose=1, steps=math.ceil(test_generator.samples/test_generator.batch_size), workers=16)
y_pred = np.argmax(Y_pred, axis=1)

validacion = dict(file=test_generator.filepaths, 
                  y=test_generator.classes, 
                  y_pred=y_pred, 
                  y_raw=np.round(Y_pred, 4).tolist())

df_val = pd.DataFrame(validacion)
df_val[f'Class_cat_{NUM_CLASSES}'] = df_val['y']
df_val[f'Class_cat_{NUM_CLASSES}_pred'] = df_val['y_pred']
class_dict = dict((v,k) for k,v in test_generator.class_indices.items())
df_val = df_val.replace({f"Class_cat_{NUM_CLASSES}": class_dict, f"Class_cat_{NUM_CLASSES}_pred": class_dict})
df_val.to_csv(os.path.join(reporte_folder, 'validacion.csv'))
df_val['correcto'] = np.where(df_val['y']==df_val['y_pred'],True, False)
incorrectos = df_val[df_val['correcto']==False]
incorrectos = pd.merge(incorrectos,datos_val, on='file', how='inner')
incorrectos.to_csv(os.path.join(reporte_folder,'incorrectos.csv'))

In [6]:
# precision normal
utils.grafica_metricas(ep, acc, val_accuracy, etiqueta='Exactitud', 
                 titulo=f'Exactitud en entrenamiento y validación para {NUM_CLASSES} clases',
                 archivo=f'{reporte_folder}/acc',
                 suavizado=False)

# precision suave
utils.grafica_metricas(ep, acc, val_accuracy, etiqueta='Exactitud', 
                 titulo=f'Exactitud en entrenamiento y validación para {NUM_CLASSES} clases (suavizado exponencial: 0.8)',
                 archivo=f'{reporte_folder}/acc_smooth',
                 suavizado=True)

# perdida normal
utils.grafica_metricas(ep, loss, val_loss, etiqueta='Pérdida', 
                 titulo=f'Pérdida en entrenamiento y validación para {NUM_CLASSES} clases',
                 archivo=f'{reporte_folder}/loss',
                 suavizado=False)
# perdida suave
utils.grafica_metricas(ep, loss, val_loss, etiqueta='Pérdida', 
                 titulo=f'Pérdida en entrenamiento y validación para {NUM_CLASSES} clases (suavizado exponencial: 0.8)',
                 archivo=f'{reporte_folder}/loss_smooth',
                 suavizado=True)

In [7]:
# variabilidad exactitud
utils.variabilidad_metricas(df_log[['accuracy', 'val_accuracy']], 'Exactitud', 
                      f'Variabilidad entre ambas exactitudes para {NUM_CLASSES} clases',
                      f'{reporte_folder}/variabilidad_acc', fontsize=10)
# variabilidad perdida
utils.variabilidad_metricas(df_log[['loss', 'val_loss']], 'Pérdida', 
                      f'Variabilidad entre ambas pérdidas para {NUM_CLASSES} clases',
                      f'{reporte_folder}/variabilidad_loss', fontsize=10)

In [8]:
utils.boxplot_metricas(df_log[['loss', 'val_loss']], 
                 f'Gráfico de caja para la pérdida en entrenamiento y en validación para {NUM_CLASSES} clases', 'tab20c',
                 f'{reporte_folder}/box_loss')

utils.boxplot_metricas(df_log[['accuracy', 'val_accuracy']], 
                 f'Gráfico de caja para las métricas en entrenamiento y en validación para {NUM_CLASSES} clases', 'tab20b',
                 f'{reporte_folder}/box_acc')

In [9]:
cm = utils.evaluar_clasificacion(df_val, NUM_CLASSES, f'reporte_{NUM_CLASSES}_class/')

In [10]:
utils.plot_confusion_matrix(cm, f'{reporte_folder}/matriz',
                      title=f'Matriz de confusion para {NUM_CLASSES} clases', 
                      normalize=False
                      )

In [11]:
utils.plot_confusion_matrix(cm, archivo=f'{reporte_folder}/matriz',
                      title=f'Matriz de confusion para {NUM_CLASSES} clases', 
                      normalize=True)

In [12]:
utils.mostrar_reporte(f'{reporte_folder}/reporte_clasificacion.csv')
utils.mostrar_reporte(f'{reporte_folder}/reporte_metricas_malas.csv')
utils.mostrar_reporte(f'{reporte_folder}/reporte_metricas_diagnostico.csv')
utils.mostrar_reporte(f'{reporte_folder}/reporte_metricas_clasificacion.csv')
utils.mostrar_reporte(f'{reporte_folder}/reporte_cero.csv')
utils.mostrar_reporte(f'{reporte_folder}/reporte_uno.csv')

In [13]:
display(HTML(filename=f'{reporte_folder}/reporte_html.html'))

In [14]:
utils.grafica_reporte_uno(NUM_CLASSES, archivo=f'{reporte_folder}/reporte_uno')
utils.grafica_reporte_cero(NUM_CLASSES, archivo=f'{reporte_folder}/reporte_cero')

In [15]:
if NUM_CLASSES == 2:
    utils.grafica_roc(df_val, NUM_CLASSES, archivo=f'{reporte_folder}/roc')
else:
    utils.roc_multiclass(NUM_CLASSES, df_val['y'].values, df_val['y_pred'].values, class_list, archivo=f'{reporte_folder}/roc_multiclase')

In [16]:
# incorrectos = pd.read_csv(os.path.join(reporte_folder, 'incorrectos.csv'))
# incorrectos.columns

In [17]:
incorrectos = incorrectos.rename(columns={"Class_cat_7_x": 'Class_cat_7'})
conteo_incorrectos = incorrectos.groupby(['Class_cat_7']).agg(['count'])
display(conteo_incorrectos)

In [18]:
muestras = []
for ele in list(conteo_incorrectos.index):
    m = incorrectos[incorrectos['Class_cat_7'] == ele].sample(n=1)
    muestras.append(m)
df_muestras = pd.concat(muestras)
df_muestras.to_csv(os.path.join(reporte_folder, 'muestreo_incorrectos.csv'))
display(df_muestras)

In [19]:
utils.imagen_muestras_erroneas(df_muestras, NUM_CLASSES, folder= f'{reporte_folder}/muestras_erroneas', font=f'../fonts/computer-modern/cmunssdc.ttf', size=25)